In [273]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor

import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [274]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all")

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
count,1253,1226.000000,1243,1253,1247,1237,1221,1241,1126,1096,1.253000e+03,829,1185,1236,1224,1237,1235,1228
unique,1248,NaN,3,119,148,48,53,24,256,562,NaN,168,45,11,3,14,5,63
top,24/11/2020 13:55:19,NaN,Male,Berlin,Software Engineer,10,2,Senior,Java,Javascript / Typescript,NaN,0,30,Full-time employee,Unlimited contract,English,1000+,Product
freq,2,NaN,1049,681,387,138,195,565,184,44,NaN,227,488,1190,1159,1020,448,760
mean,NaN,32.509788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.027904e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,5.663804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.825061e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,20.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000100e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,29.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.880000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,32.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.000000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.000000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [275]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 18 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Timestamp                                               1253 non-null   object 
 1   Age                                                     1226 non-null   float64
 2   Gender                                                  1243 non-null   object 
 3   City                                                    1253 non-null   object 
 4   Position                                                1247 non-null   object 
 5   Total years of experience                               1237 non-null   object 
 6   Years of experience in Germany                          1221 non-null   object 
 7   Seniority level                                         1241 non-null   object 
 8   Your main technology / programming lan

In [276]:
df.isna().sum()

Timestamp                                                   0
Age                                                        27
Gender                                                     10
City                                                        0
Position                                                    6
Total years of experience                                  16
Years of experience in Germany                             32
Seniority level                                            12
Your main technology / programming language               127
Other technologies/programming languages you use often    157
Yearly brutto salary (without bonus and stocks) in EUR      0
Yearly bonus + stocks in EUR                              424
Number of vacation days                                    68
Employment status                                          17
Сontract duration                                          29
Main language at work                                      16
Company 

In [277]:
df[393:396]

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
393,24/11/2020 19:15:02,30.0,Male,Moscow,Software Engineer,5,0,Middle,C,C/C++,14712.0,0,30,Full-time employee,Unlimited contract,Russian,101-1000,Product
394,24/11/2020 19:15:49,33.0,Male,Berlin,Product Manager,5,5,Senior,NaN,Python,70000.0,800,30,Full-time employee,NaN,German,101-1000,Product
395,24/11/2020 19:19:30,35.0,Male,Berlin,QA Engineer,11,10,Senior,Java,"Python, Javascript / Typescript, .NET, Java / ...",74400.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Product


In [278]:
# Construct Target by adding salary and bonus. 
df["Bonus"] = pd.to_numeric(df["Yearly bonus + stocks in EUR"], downcast="float", errors="coerce")
df["Bonus"].fillna(0, inplace=True)
df["target"] = df["Yearly brutto salary (without bonus and stocks) in EUR"] + df["Bonus"]
df.drop(columns={"Timestamp","Yearly brutto salary (without bonus and stocks) in EUR", "Yearly bonus + stocks in EUR", "Bonus"}, inplace=True)
df.head()

,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type,target
0,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",30,Full-time employee,Unlimited contract,English,51-100,Product,85000.0
1,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product,80000.0
2,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",30,Self-employed (freelancer),Temporary contract,English,101-1000,Product,240000.0
3,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup,54000.0
4,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",29,Full-time employee,Unlimited contract,English,101-1000,Product,62000.0


Data Cleaning



In [279]:
#since its not a big amount of missing data replace 27 missing age with mean
df['Age'] = df['Age'].fillna(value = df['Age'].mean())

#Since Gender only has 10 missing values, we can use empty with the common one
df['Gender'] = df['Gender'].fillna(value = df['Gender'].mode().sum()) # mode only works if you add .sum note for myself

#Position s/a gender
df['Position '] = df['Position '].fillna(value = df['Position '].mode().sum()) #Make sure Position df name includes space after Position. Weird naming ngl

#Total years contains NaN so will have to work around it, i also saw max experience as 383 which doesnt make sense so we shall limit it
df['Total years of experience'] = pd.to_numeric(df['Total years of experience'], downcast = "float", errors = "coerce")
df['Total years of experience'] = df['Total years of experience'].replace(np.nan,df['Total years of experience'].mean())
df = df[df['Total years of experience'] <= 60] 

# Years in Germany s/a Total years 
df['Years of experience in Germany'] = pd.to_numeric(df['Years of experience in Germany'], downcast = "float", errors = "coerce")
df['Years of experience in Germany'] = df['Years of experience in Germany'].replace(np.nan,df['Years of experience in Germany'].mean())

#seniroity s/a gender
df['Seniority level'] = df['Seniority level'].fillna(value = df['Seniority level'].mode().sum())

#Company size nulls
df['Company size'] = df['Company size'].fillna(value = df['Company size'].mode().sum())

#Company Type
df['Company type'] = df['Company type'].fillna(value = df['Company type'].mode().sum())

#Columns to drop
df.drop(['Your main technology / programming language', 'Other technologies/programming languages you use often', 'Number of vacation days', 'Employment status', 'Сontract duration', 'Main language at work'  ], axis = 1, inplace = True)

In [280]:
#df.info()
df.isna().sum()

Age                               0
Gender                            0
City                              0
Position                          0
Total years of experience         0
Years of experience in Germany    0
Seniority level                   0
Company size                      0
Company type                      0
target                            0
dtype: int64

No more nulls so we can go ahead and continue with the model

In [281]:
df.sample(20)

,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Company size,Company type,target
716,26.0,Male,London,DevOps,3.0,0.000000,Middle,51-100,Product,37000.0
1190,37.0,Male,Aachen,QA Engineer,15.0,1.000000,Senior,101-1000,Product,77000.0
167,28.0,Male,Moldova,Backend Developer,4.0,0.000000,Middle,51-100,Startup,28000.0
849,42.0,Male,Berlin,Software Engineer,12.0,12.000000,Lead,101-1000,Product,95000.0
1069,26.0,Male,Munich,Software Engineer,6.0,2.000000,Senior,11-50,Startup,103000.0
261,38.0,Male,Berlin,Backend Developer,11.0,1.000000,Senior,up to 10,Startup,35000.0
1127,35.0,Male,Duesseldorf,DevOps,17.0,5.000000,Senior,1000+,Consulting / Agency,87000.0
673,51.0,Male,Berlin,Software Engineer,30.0,30.000000,Senior,1000+,Consulting / Agency,65000.0
1050,28.0,Male,Munich,Software Engineer,8.0,1.000000,Senior,51-100,Product,57000.0
18,27.0,Male,Berlin,Backend Developer,8.0,3.000000,Senior,1000+,Product,60000.0


In [282]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1252 entries, 0 to 1252
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Age                             1252 non-null   float64
 1   Gender                          1252 non-null   object 
 2   City                            1252 non-null   object 
 3   Position                        1252 non-null   object 
 4   Total years of experience       1252 non-null   float32
 5   Years of experience in Germany  1252 non-null   float32
 6   Seniority level                 1252 non-null   object 
 7   Company size                    1252 non-null   object 
 8   Company type                    1252 non-null   object 
 9   target                          1252 non-null   float64
dtypes: float32(2), float64(2), object(6)
memory usage: 97.8+ KB


# Pipeline

In [283]:
#from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


numeric_features = ["Age", "Total years of experience", "Years of experience in Germany"]
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_features = ["Gender", "City", "Position ", "Seniority level", "Company size", "Company type"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])


In [284]:
y = df["target"].values.reshape(-1, 1)
X = df.drop(columns={"target"})

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Build pipeline
l1_model = Lasso()
pipeline_steps = [('pre', preprocessor), ('model', l1_model)]
pipe = Pipeline(pipeline_steps)

pipe.fit(X_train, y_train.ravel())
cvs = cross_val_score(pipe, X_train, y_train.ravel(), cv=5)
print('Training CrossVal Score:', cvs)
print("Average Train Score:", np.mean(cvs))
print('Testing score:', pipe.score(X_test, y_test))


c:\Users\drigs\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.310370881321531e+21, tolerance: 1.0018378353709631e+18
  model = cd_fast.sparse_enet_coordinate_descent(


c:\Users\drigs\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.1685198936060057e+21, tolerance: 9.986667010829226e+17
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\drigs\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.1633614393198914e+21, tolerance: 1.001541123681633e+18
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\drigs\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.22586691171845e+21, tolerance: 1.0015411057099852e+18
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\drigs\anaconda3\Lib\site-packages\sklearn\linear_mode

Training CrossVal Score: [-1.67420779e+00 -2.51872387e+07 -1.03394018e+08 -6.34838428e-03
 -4.97310175e+07]
Average Train Score: -35662455.119838335
Testing score: -94389612.21216674


c:\Users\drigs\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.117861477115703e+21, tolerance: 1.0015430794276499e+18
  model = cd_fast.sparse_enet_coordinate_descent(


Im not sure how to pick a model was going to study during the break but got sick. I think my employees are all going to be broke once their Bonus comes out lol

# Answers and Explainations
(Expand/modify as needed)

### Results

Initial EDA Resulted in multiple nulls which I ended up cleaning. 
Data Cleaning only on datasets that I think will result on a more accurate model


### Feature Selection Activities

<li>Technology / language (Together with the other technology)
<br>Initially I thought Programming language can affect the salary but the more I think about it it doesnt. Majority of the languages can be learned, having multiple language can be a asset but shouldnt be the main compensation basis. Maybe Ill test the Main technology since I still think that having a MAIN language or specialty can differentiate you depending on the position, but on the general scope I believe it shouldnt. Another part of my reasoning is saturation. There are many skilled workers or potential applicants on each language, either coming from old tech jobs or learning new ones. However I might be wrong. 

<li>Number of vacation days
<br> The seniority is what affects this column the most so I believe its reasonable to ignore this plus we are predicting total comp not the vacation days 

<li>Employment status
<br> I dont think the employment status should affect the prediction as well so we can drop this column 

<li>Main language at work
<br> This shouldnt matter as far as im aware unless youre in marketing? I might be wrong 

### Hyperparameter Changes